In [1]:
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome(r"C:\Users\ASUS\chromedriver.exe")

In [4]:
# This function is define to take input and generate a range of date's
def start_date(year,month,day,no_of_days):# Enter year, month and day from when you want to start scrapping
    date_range =[]
    import datetime
    from datetime import date
    date = datetime.date(year, month, day)
    str_date = date.strftime("%Y-%m-%d")

    date = pd.date_range(start=str_date,periods=no_of_days)# You can change the the periods to extract more no. of days data and frequency
    for i in date:
        date_range.append('{:%Y-%m-%d}'.format(i))# Converting date into proper format
    return date_range

'''Here i am fetching the data with frequency of 6 days. So that every week price can cover init'''
date_range = start_date(2021,11,28,7)# Pass year, month,day and no_of_days here

In [5]:
source=[]
destination=[]
date_of_journey=[]
fare =[]
duration =[]
arrival_time = []
departure_time= []
stops =[]
Name=[]


# Passing locations in list
boarding = ["BLR-Bengaluru","PNQ-Pune","MAA-Chennai","GOI-Goa"]
destina=["PNQ-Pune","GOI-Goa","BOM-Mumbai","DEL-Delhi","CCU-Kolkata","COK-Kochi"]
for i in range(0,4):#Using range function to pass one source at a time and checking it with all the other places accept itself
    
    for j in range(0,6):# Using loop in a loop so that all the possible combination of source and destination can form
        
        if boarding[i] != destina[j]:# Make sure that both source and destination should not be same
            
            for k in date_range:# passing date 
                url = "https://paytm.com/flights/flightSearch/{}/{}/1/0/0/E/{}".format(boarding[i],destina[j],k)
                
                driver.get(url)# Opening url
                time.sleep(6)
                
                # Departure time from source
                for l in driver.find_elements_by_xpath('//div[@class="_3H-S"]'):
                    departure_time.append(l.text)
                # Arrival time at destination
                for l in driver.find_elements_by_xpath('//div[@class="_3H-S _1wD5"]'):
                    arrival_time.append(l.text.split("\n")[0])
                # Total duration    
                for l in driver.find_elements_by_xpath('//div[@class="vY4t"]'):
                    duration.append(l.text)
                # Price of the flight ticket    
                for l in driver.find_elements_by_xpath('//div[@class="_1cxG"]'):
                    fare.append(int(l.text.replace(",","")))
                # Total stop    
                for stop in driver.find_elements_by_xpath('//div[@class="_7BOG"]'):
                    stops.append(stop.text)
                #Fetching Name            
                for name in driver.find_elements_by_xpath('//div[@class="_3H-S _1Eia"]'):
                    Name.append(name.text)
            
       
    # Fetching date, source and destination and replicate it to the  times of flight name present on the current page
                for rem in range(0,len(driver.find_elements_by_xpath('//div[@class="_3H-S _1Eia"]'))):
                    date_of_journey.append(url.split("/")[-1])
                    source.append(url.split("/")[-7])
                    destination.append(url.split("/")[-6])

In [6]:
print(len(Name),len(departure_time),len(arrival_time),len(destination),len(source),len(fare),len(stops),len(duration),len(date_of_journey))

14697 14697 14697 14697 14697 14697 14697 14697 14697


In [12]:
df = pd.DataFrame({"Name":Name,"Source":source,"destination":destination,"date_of_journey":date_of_journey,"fare":fare,
                  "duration":duration,"arrival_time":arrival_time,"departure_time":departure_time,"stops":stops})

In [13]:
df.to_csv("Final_dataset.csv")